In [49]:
import utils.age_processing as ap
import utils.genre_processing as gp
import os
import ast
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import entropy, ttest_ind
import numpy as np
from scipy.spatial.distance import jensenshannon
matplotlib.use('TkAgg')

In [50]:
from dotenv import load_dotenv
from pathlib import Path
env_path = Path('..') / 'config.env'
load_dotenv(dotenv_path=env_path)
dataset_dir = os.getenv("dataset_directory")

In [51]:
dataset = 'mlhd' # 'ml', 'mlhd', or 'bx'
weighted = True
age_type = 'defined_ages' # 'defined_ages', 'even_age_groups', 'finegrained_age', 'finegrained_child_ages'

In [52]:
ages_sort = ap.get_sorted_ages(dataset, age_type)

In [53]:
genres = []

if dataset == 'ml':
    data_dir = dataset_dir + '/processed/movielens-1m'
    with open('../utils/ML_genres.txt', 'r') as f:
        for line in f:
            genres.append(line.strip())
elif dataset == 'mlhd':
    data_dir = dataset_dir + '/processed/MLHD_sampled_filtered'
    with open('../utils/MLHD_genres.txt', 'r') as f:
        for line in f:
            genres.append(line.strip())
elif dataset == 'bx':
    data_dir = dataset_dir + '/processed/Book-Crossing'
    with open('../utils/BX_genres.txt', 'r') as f:
        for line in f:
            genres.append(line.strip())

In [55]:
user_profile_stats_path = data_dir + f'/user_profile_stats{"_weighted" if weighted else ""}.tsv'
user_path = data_dir + '/users.tsv'
interactions_path = data_dir + '/interactions.tsv.bz2'

#users = pd.read_csv(user_path, sep='\t')
user_stats = pd.read_csv(user_profile_stats_path, sep='\t')
user_stats['normalized_genre_distribution'] = user_stats['normalized_genre_distribution'].apply(ast.literal_eval)

user_stats['age_group'] = user_stats['age'].apply(lambda x: ap.age_group(x, dataset, age_type))
user_stats['age_group'] = pd.Categorical(user_stats['age_group'], categories=ages_sort, ordered=True)

grouped_user_stats = user_stats.groupby('age_group')

C:\Users\rungruh\AppData\Local\Temp\ipykernel_6360\4106223412.py:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_user_stats = user_stats.groupby('age_group')


In [56]:
user_stats.columns

Index(['user_id', 'age', 'num_interactions', 'num_unique_items',
       'normalized_genre_distribution', 'age_group'],
      dtype='object')

In [57]:
genres = []

if dataset == 'mlhd':
    genres = gp.MLHD_genres
elif dataset == 'ml':
    genres = gp.ML_genres
elif dataset == 'bx':
    genres = gp.BX_genres
    


age_genre_profiles = pd.DataFrame(columns=('age_group', 'genre_distribution'))
print("Calculating genre distributions for each age group...")

for age, group in grouped_user_stats:
    genre_sum = {}
    num_users = len(group)
    for i,user in group.iterrows():

        for user_genre, value in user['normalized_genre_distribution'].items():
            if user_genre in genre_sum:
                genre_sum[user_genre] += value
            else:
                genre_sum[user_genre] = value
       
    genre_avg = {genre: g_sum / num_users for genre, g_sum in genre_sum.items()}
    if age in age_genre_profiles['age_group'].values:
        age_genre_profiles.loc[age_genre_profiles['age_group'] == age, 'genre_distribution'] = [genre_avg]
        print("this should not happen") # test statement
    else:
        age_genre_profiles = pd.concat([age_genre_profiles, pd.DataFrame({'age_group': [age], 'genre_distribution': [genre_avg]})], ignore_index=True)



age_genre_profiles['genre_distribution'] = age_genre_profiles['genre_distribution'].apply(lambda x: gp.genre_dict_to_list(x, dataset))

user_stats['genre_distribution'] = user_stats['normalized_genre_distribution'].apply(lambda x: gp.genre_dict_to_list(x, dataset))

grouped_user_stats = user_stats.groupby('age_group')

age_genre_profiles['age_group'] = pd.Categorical(age_genre_profiles['age_group'], 
                                            categories=ages_sort, 
                                            ordered=True)

    
age_genre_profiles.sort_values('age_group', inplace=True)

Calculating genre distributions for each age group...


C:\Users\rungruh\AppData\Local\Temp\ipykernel_6360\2617787729.py:39: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_user_stats = user_stats.groupby('age_group')


In [58]:
plot_genres = [genre.capitalize() for genre in genres]
# Prepare data for stacked bar plot
genre_data = pd.DataFrame(age_genre_profiles['genre_distribution'].tolist(), index=age_genre_profiles['age_group'], columns=plot_genres)




# Plot stacked bar chart
plt.figure(figsize=(8, 6))
sns.set(style='whitegrid')
genre_data.plot(kind='bar', stacked=True, cmap='tab20', ax=plt.gca())
plt.xlabel('')
plt.ylabel('Average Genre Distribution', fontsize=16)
handles, labels = plt.gca().get_legend_handles_labels() 
labels = ['/'.join([l.capitalize() for l in label.split('_')]) for label in labels]
#plt.legend(title='Genre', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=12)
plt.legend(handles[::-1], labels[::-1], loc='lower center', bbox_to_anchor=(0.5, 0.97),
          ncol=5, fancybox=True, shadow=True, fontsize=10)
plt.xticks(rotation=45, fontsize=16)
plt.yticks(fontsize=16)
plt.tight_layout()
plt.savefig(f'Results/genre_distribution_{dataset}_{age_type}.pdf', bbox_inches='tight')
plt.show()

In [59]:
epsilon = 1e-10
print("Calculating average Jensen-Shannon Divergence within age groups...")
# Initialize a DataFrame to store the average JSD values for each age group
intra_group_js_divergences = pd.DataFrame(columns=['age_group', 'average_js_divergence'])

from scipy.spatial.distance import jensenshannon

# Iterate over each age group
for age, group in grouped_user_stats:
    print(f"Processing age group: {age}")
    num_users = len(group)
    js_divergences = []
    # Transform genre distributions for vectorized operations
    genre_distributions = np.array([np.array(user['genre_distribution']) + epsilon for i, user in group.iterrows()])
    
    avg_age_genre_distribution = np.mean(genre_distributions, axis=0)
    
    # Calculate pairwise JSD for all users within the same age group
    pairwise_jsd = [jensenshannon(avg_age_genre_distribution, user_genre_distribution)**2 for user_genre_distribution in genre_distributions]

    pairwise_jsd = [0 if np.isnan(jsd) else jsd for jsd in pairwise_jsd]
            
    # Average the pairwise JSDs
    average_js_divergence = np.mean(pairwise_jsd) if len(pairwise_jsd) > 0 else 0
    intra_group_js_divergences = pd.concat([intra_group_js_divergences, pd.DataFrame({'age_group': [age], 'average_js_divergence': [average_js_divergence]})], ignore_index=True)

# Sort the DataFrame by age
intra_group_js_divergences['age_group'] = pd.Categorical(intra_group_js_divergences['age_group'], categories=ages_sort, ordered=True)
intra_group_js_divergences.sort_values('age_group', inplace=True)

# Plot the results
plt.figure(figsize=(8, 6))
sns.barplot(data=intra_group_js_divergences, x='age_group', y='average_js_divergence', palette='tab20')
plt.ylabel('Distance', fontsize=16)
plt.xlabel('')
plt.xticks(rotation=45, fontsize=16)
plt.yticks(fontsize=16)
plt.tight_layout()
plt.savefig(f'Results/in_group_jsd_{dataset}_{age_type}.pdf', bbox_inches='tight')
plt.show()

Calculating average Jensen-Shannon Divergence within age groups...
Processing age group: 12-16


C:\Users\rungruh\AppData\Local\Temp\ipykernel_6360\571959192.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  intra_group_js_divergences = pd.concat([intra_group_js_divergences, pd.DataFrame({'age_group': [age], 'average_js_divergence': [average_js_divergence]})], ignore_index=True)


Processing age group: 17-29
Processing age group: 30-65


C:\Users\rungruh\AppData\Local\Temp\ipykernel_6360\571959192.py:33: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(data=intra_group_js_divergences, x='age_group', y='average_js_divergence', palette='tab20')


In [60]:
print("Calculating Jensen-Shannon Divergence between age groups...")
inter_group_js_divergences = {}
epsilon = 1e-10

for i, row1 in age_genre_profiles.iterrows():
    age1 = row1['age_group']
    genre_distribution1 = np.array(row1['genre_distribution']) + epsilon
    
    for j, row2 in age_genre_profiles.iterrows():
        age2 = row2['age_group']
        genre_distribution2 = np.array(row2['genre_distribution']) + epsilon

        # Jensen-Shannon Divergence
        js_div = jensenshannon(genre_distribution1, genre_distribution2)**2
        inter_group_js_divergences[(age1, age2)] = js_div

ages = age_genre_profiles['age_group'].unique()  

heatmap_data = pd.DataFrame(index=ages, columns=ages)
for (age1, age2), js_div in inter_group_js_divergences.items():
    #print(f"Jensen-Shannon Divergence between age {age1} and age {age2}: {js_divergence}")
    heatmap_data.at[age1, age2] = js_div
    heatmap_data.at[age2, age1] = js_div  # Ensure symmetry
    
heatmap_data = heatmap_data.astype(float)
# Plot the heatmap
plt.figure(figsize=(8, 6))
ax = sns.heatmap(heatmap_data, annot=True, cmap="coolwarm_r", linewidths=0.5, cbar=False, annot_kws={"size": 16})
#plt.title("Jensen-Shannon Divergence Between Age Groups")

plt.xlabel("")
plt.ylabel("")
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontsize=16)
ax.set_yticklabels(ax.get_yticklabels(), rotation=0, ha='right', fontsize=16)  # Set rotation to 0 for horizontal

plt.tight_layout()
plt.savefig(f'Results/age_preference_deviations_{dataset}_{age_type}.pdf', bbox_inches='tight')
plt.show()

Calculating Jensen-Shannon Divergence between age groups...


In [61]:
#### This was used in the original study

if not age_type == 'defined_ages':
    raise ValueError("This analysis is only valid for defined_ages age type.")
else:
    print("Performing ANOVA analysis on genre distributions...")

    genres = [genre.replace("'", "").replace("-", "").replace(" ", "") for genre in genres]
    genre_df = pd.DataFrame(user_stats['genre_distribution'].apply(np.array).tolist())
    genre_df.columns = genres
    genre_df = genre_df.astype(float)    

    # Combine with the original DataFrame
    genre_df = pd.concat([user_stats['age_group'], genre_df], axis=1)
    

    # Drop nma users
    genre_df = genre_df[~genre_df['age_group'].isin([ages_sort[-1]])]
    genre_df['age_group'] = genre_df['age_group'].astype(str)

    
    #print(genre_df.head(n=3))
    from statsmodels.multivariate.manova import MANOVA
    
    # # Fit the MANOVA model
    manova = MANOVA.from_formula(' + '.join(genres) + ' ~ age_group', data=genre_df)
    manova_results = manova.mv_test()



    
    from statsmodels.stats.multicomp import pairwise_tukeyhsd

    from statsmodels.stats.multicomp import pairwise_tukeyhsd

    # Melt the DataFrame to long format for ANOVA
    melted_df = genre_df.melt(id_vars=['age_group'], value_vars=genre_df.columns, var_name='genre', value_name='value')


    import statsmodels.api as sm
    from statsmodels.formula.api import ols

    # Perform ANOVA for each genre
    anova_results = {}
    for genre in genre_df.drop('age_group', axis=1).columns:
        model = ols(f'{genre} ~ age_group', data=genre_df).fit()
        anova_table = sm.stats.anova_lm(model, typ=2)
        anova_results[genre] = anova_table

    # Print ANOVA results for each genre
    for genre, result in anova_results.items():
        if result['PR(>F)'].iloc[0] < 0.05:
            print(f'ANOVA results for {genre}:')
            print(result)
            print('\n')

    # Tukey's HSD test for each genre
    tukey_results = {}
    for genre in genres:
        tukey = pairwise_tukeyhsd(endog=melted_df[melted_df['genre'] == genre]['value'],
                                groups=melted_df[melted_df['genre'] == genre]['age_group'],
                                alpha=0.01)
        tukey_results[genre] = tukey

    print('sigificant Genres')
    # Print Tukey's HSD results for each genre
    for genre, result in tukey_results.items():
        if result.reject.any():
            
            print(f'Tukey HSD results for {genre}:')
            print(result)
            print('\n')

    print('non-significant Genres')
    for genre, result in tukey_results.items():
        if not result.reject.any():
            print(f'Tukey HSD results for {genre}:')
            print(result)
            print('\n')
        

Performing ANOVA analysis on genre distributions...
ANOVA results for rnb:
               sum_sq        df          F        PR(>F)
age_group    0.104998       1.0  64.846002  8.144209e-16
Residual   312.470948  192979.0        NaN           NaN


ANOVA results for rap:
               sum_sq        df        F        PR(>F)
age_group    0.255505       1.0  99.7918  1.715312e-23
Residual   494.098980  192979.0      NaN           NaN


ANOVA results for electronic:
                sum_sq        df            F         PR(>F)
age_group    14.878783       1.0  1077.022426  1.460132e-235
Residual   2665.954426  192979.0          NaN            NaN


ANOVA results for rock:
                sum_sq        df           F        PR(>F)
age_group     1.770320       1.0  156.723513  6.075104e-36
Residual   2179.855656  192979.0         NaN           NaN


ANOVA results for newage:
              sum_sq        df           F        PR(>F)
age_group   0.028733       1.0  194.317726  3.813877e-44
Resi

In [62]:
from scipy.stats import kruskal, mannwhitneyu
from itertools import combinations
from statsmodels.stats.multitest import multipletests
import pandas as pd

if not age_type == 'defined_ages':
    raise ValueError("This analysis is only valid for defined_ages age type.")
else:
    genres = [genre.replace("'", "").replace("-", "").replace(" ", "") for genre in genres]
    genre_df = pd.DataFrame(user_stats['genre_distribution'].apply(np.array).tolist())
    genre_df.columns = genres
    genre_df = genre_df.astype(float)    

    # Combine with the original DataFrame
    genre_df = pd.concat([user_stats['age_group'], genre_df], axis=1)
    

    # Drop nma users
    # genre_df = genre_df[~genre_df['age_group'].isin([ages_sort[-1]])]
    # genre_df['age_group'] = genre_df['age_group'].astype(str)
    
        
    # Perform Kruskal–Wallis test for each genre
    kruskal_results = []
    significant_genres = []

    for genre in genre_df.drop('age_group', axis=1).columns:
        grouped = [group[genre].values for _, group in genre_df.groupby('age_group')]
        stat, p = kruskal(*grouped)
        kruskal_results.append({'genre': genre, 'H-stat': stat, 'p-value': p})
        if p < 0.05:
            significant_genres.append(genre)

    kruskal_df = pd.DataFrame(kruskal_results).sort_values('p-value')

    # Print significant Kruskal–Wallis results
    print('Significant Kruskal–Wallis genres (p < 0.05):')
    print(kruskal_df[kruskal_df['p-value'] < 0.05])
    print('\n')

    # Pairwise Mann–Whitney U tests for significant genres
    posthoc_results = []

    for genre in significant_genres:
        for g1, g2 in combinations(genre_df['age_group'].unique(), 2):
            data1 = genre_df[genre_df['age_group'] == g1][genre]
            data2 = genre_df[genre_df['age_group'] == g2][genre]
            stat, p = mannwhitneyu(data1, data2, alternative='two-sided')
            posthoc_results.append({
                'genre': genre,
                'group1': g1,
                'group2': g2,
                'U-stat': stat,
                'p-raw': p
            })

    # Multiple testing correction (FDR)
    posthoc_df = pd.DataFrame(posthoc_results)
    posthoc_df['p-corrected'] = multipletests(posthoc_df['p-raw'], method='fdr_bh')[1]

    # Split into significant and non-significant
    print('Significant Mann–Whitney U posthoc comparisons (FDR < 0.05):')
    print(posthoc_df[posthoc_df['p-corrected'] < 0.05])
    print('\n')

    print('Non-significant Mann–Whitney U posthoc comparisons:')
    print(posthoc_df[posthoc_df['p-corrected'] >= 0.05])

C:\Users\rungruh\AppData\Local\Temp\ipykernel_6360\854725043.py:28: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = [group[genre].values for _, group in genre_df.groupby('age_group')]


Significant Kruskal–Wallis genres (p < 0.05):
            genre       H-stat        p-value
1             rap  2718.304527   0.000000e+00
2      electronic  1799.398950   0.000000e+00
5       classical  2893.059895   0.000000e+00
4          newage  4118.307322   0.000000e+00
7           blues  3917.545101   0.000000e+00
10           folk  4070.594384   0.000000e+00
9           world  3084.008817   0.000000e+00
8         country  5231.957366   0.000000e+00
12           jazz  8232.991760   0.000000e+00
13          vocal  3900.118292   0.000000e+00
14      childrens  2488.492892   0.000000e+00
11  easylistening  3439.363929   0.000000e+00
17     spokenword  2690.075760   0.000000e+00
16    alternative  3426.483257   0.000000e+00
19     heavymetal  1411.518932  3.108554e-307
15           punk  1328.529688  3.261698e-289
0             rnb   660.136228  4.500279e-144
6          reggae   415.344491   6.443024e-91
3            rock   258.035540   9.295966e-57
18            pop    49.379489   1

# Popularity Analysis

In [ ]:
# ['user_id', 'age', 'num_interactions', 'num_unique_items', 'avg_popularity', 'avg_normalized_popularity', 'avg_age_group_popularity', 'avg_normalized_age_group_popularity', 'age_group']

for variable in ['num_interactions', 'num_unique_items', 'avg_popularity', 'avg_normalized_popularity', 'avg_age_group_popularity', 'avg_normalized_age_group_popularity']:
    # Paired t-test for each variable
    print(f'Mean {variable} for each age group:')
    print(user_stats.groupby('age_group')[variable].mean().round(3))
    for age_group_1 in ages_sort:
        for age_group_2 in ages_sort:
            if age_group_1 != age_group_2:
                group_1 = user_stats[user_stats['age_group'] == age_group_1][variable]
                group_2 = user_stats[user_stats['age_group'] == age_group_2][variable]

                # Perform independent t-test
                t_stat, p_value = ttest_ind(group_1, group_2, equal_var=False)
                print(f"Independent t-test for {variable} between {age_group_1} and {age_group_2}:")
                print(f"t-statistic: {t_stat}, p-value: {p_value}")
                print('\n')
                
    print("\n\n")
    print("--------------------------------------------------")
    print("\n\n")
                

C:\Users\rungruh\AppData\Local\Temp\ipykernel_6360\61298322.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(user_stats.groupby('age_group')[variable].mean().round(3))


Mean num_interactions for each age group:
age_group
12-17     4.508
18-49    11.949
50-65    11.360
Name: num_interactions, dtype: float64
Independent t-test for num_interactions between 12-17 and 18-49:
t-statistic: -14.779424797798255, p-value: 6.13675890722638e-49


Independent t-test for num_interactions between 12-17 and 50-65:
t-statistic: -5.362708994661505, p-value: 8.488770596012577e-08


Independent t-test for num_interactions between 18-49 and 12-17:
t-statistic: 14.779424797798255, p-value: 6.13675890722638e-49


Independent t-test for num_interactions between 18-49 and 50-65:
t-statistic: 0.45796669335003504, p-value: 0.6469915551014715


Independent t-test for num_interactions between 50-65 and 12-17:
t-statistic: 5.362708994661505, p-value: 8.488770596012577e-08


Independent t-test for num_interactions between 50-65 and 18-49:
t-statistic: -0.45796669335003504, p-value: 0.6469915551014715





--------------------------------------------------



Mean num_unique_items f

In [ ]:
plt.figure(figsize=(12, 8))
sns.barplot(data=user_stats, x='age_group', y='avg_normalized_popularity', palette='tab10')
plt.xlabel('Age Group', fontsize=16)
plt.ylabel('Average Normalized Popularity', fontsize=16)
plt.xticks(rotation=45, fontsize=14)
plt.yticks(fontsize=14)
plt.tight_layout()
plt.show()

C:\Users\rungruh\AppData\Local\Temp\ipykernel_6360\775158258.py:2: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(data=user_stats, x='age_group', y='avg_normalized_popularity', palette='tab10')
